In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 7
verbosity = 2
input_shape = (width, height, channel)

In [3]:
pretrained_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')


pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.


In [5]:
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    verbose=verbosity,
                   )

Epoch 1/7
40/40 - 22s - loss: 0.7404 - auc: 0.8527 - accuracy: 0.7790
Epoch 2/7
40/40 - 22s - loss: 0.1043 - auc: 0.9929 - accuracy: 0.9622
Epoch 3/7
40/40 - 22s - loss: 0.0447 - auc: 0.9998 - accuracy: 0.9921
Epoch 4/7
40/40 - 22s - loss: 0.0323 - auc: 1.0000 - accuracy: 0.9972
Epoch 5/7
40/40 - 23s - loss: 0.0221 - auc: 1.0000 - accuracy: 0.9992
Epoch 6/7
40/40 - 22s - loss: 0.0168 - auc: 1.0000 - accuracy: 1.0000
Epoch 7/7
40/40 - 23s - loss: 0.0128 - auc: 1.0000 - accuracy: 1.0000


In [ ]:
model.evaluate(valid_generator)

In [6]:
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

prediction = model.predict(valid_generator)

Found 650 images belonging to 2 classes.


In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

320
330
